In [1]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:\Users\mohab\OneDrive\Desktop\NLP")
CLIPS = BASE / "clips"

def check(split):
    tsv = BASE / f"{split}.tsv"
    df = pd.read_csv(tsv, sep="\t")
    print(f"\n=== {split}.tsv ===")
    print("rows:", len(df))
    print("columns:", list(df.columns))

    # must-have cols
    assert "path" in df.columns, "Missing column: path"
    assert "sentence" in df.columns, "Missing column: sentence"

    # check audio existence
    missing = []
    for p in df["path"].astype(str).head(2000):  # sample first 2000 for speed
        if not (CLIPS / p).exists():
            missing.append(p)
    print("audio missing (sample):", len(missing))
    if missing[:5]:
        print("examples:", missing[:5])

    # transcript empties
    empty = df["sentence"].isna() | (df["sentence"].astype(str).str.strip() == "")
    print("empty sentences:", int(empty.sum()))

for s in ["train","dev","test"]:
    check(s)

print("\nclips folder exists:", CLIPS.exists())
print("num audio files in clips:", len(list(CLIPS.glob('*'))))




=== train.tsv ===
rows: 196006
columns: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment']
audio missing (sample): 0
empty sentences: 0

=== dev.tsv ===
rows: 15338
columns: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment']
audio missing (sample): 0
empty sentences: 0

=== test.tsv ===
rows: 15338
columns: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment']
audio missing (sample): 0
empty sentences: 0

clips folder exists: True
num audio files in clips: 490505


In [2]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:\Users\mohab\OneDrive\Desktop\NLP")
CLIPS = BASE / "clips"

train_df = pd.read_csv(BASE/"train.tsv", sep="\t")
dev_df   = pd.read_csv(BASE/"dev.tsv",   sep="\t")
test_df  = pd.read_csv(BASE/"test.tsv",  sep="\t")

print(len(train_df), len(dev_df), len(test_df))
print(train_df.columns)
print("Example:", train_df.loc[0, ["path","sentence"]].to_dict())


196006 15338 15338
Index(['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age',
       'gender', 'accent', 'locale', 'segment'],
      dtype='object')
Example: {'path': 'common_voice_es_22140775.mp3', 'sentence': 'Esto le llevó a Holanda, Inglaterra, Francia e Italia.'}


In [3]:
import re, unicodedata

def normalize_es(text: str) -> str:
    text = str(text).lower().strip()
    text = unicodedata.normalize("NFKC", text)
    # keep letters (including accents), numbers, and spaces
    text = re.sub(r"[^\wáéíóúüñ\s]", " ", text, flags=re.UNICODE)
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [4]:
print("Before:", train_df["sentence"].iloc[0])
print("After :", normalize_es(train_df["sentence"].iloc[0]))


Before: Esto le llevó a Holanda, Inglaterra, Francia e Italia.
After : esto le llevó a holanda inglaterra francia e italia


In [5]:
(log-Mel or MFCC features, transcript)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1353091842.py, line 1)

In [6]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:\Users\mohab\OneDrive\Desktop\NLP")
CLIPS = BASE / "clips"

train_df = pd.read_csv(BASE/"train.tsv", sep="\t")
dev_df   = pd.read_csv(BASE/"dev.tsv",   sep="\t")
test_df  = pd.read_csv(BASE/"test.tsv",  sep="\t")

print("sizes:", len(train_df), len(dev_df), len(test_df))
print("example:", train_df.loc[0, ["path","sentence"]].to_dict())


sizes: 196006 15338 15338
example: {'path': 'common_voice_es_22140775.mp3', 'sentence': 'Esto le llevó a Holanda, Inglaterra, Francia e Italia.'}


In [7]:
import re, unicodedata

def normalize_es(text: str) -> str:
    text = str(text).lower().strip()
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"[^\wáéíóúüñ\s]", " ", text, flags=re.UNICODE)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# quick test
print("Before:", train_df["sentence"].iloc[0])
print("After :", normalize_es(train_df["sentence"].iloc[0]))


Before: Esto le llevó a Holanda, Inglaterra, Francia e Italia.
After : esto le llevó a holanda inglaterra francia e italia


In [8]:
def wer(ref: str, hyp: str) -> float:
    ref_words = normalize_es(ref).split()
    hyp_words = normalize_es(hyp).split()

    # edit distance
    dp = [[0]*(len(hyp_words)+1) for _ in range(len(ref_words)+1)]
    for i in range(len(ref_words)+1): dp[i][0] = i
    for j in range(len(hyp_words)+1): dp[0][j] = j

    for i in range(1, len(ref_words)+1):
        for j in range(1, len(hyp_words)+1):
            cost = 0 if ref_words[i-1] == hyp_words[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,      # deletion
                dp[i][j-1] + 1,      # insertion
                dp[i-1][j-1] + cost  # substitution
            )
    return dp[-1][-1] / max(1, len(ref_words))

# quick sanity
print(wer("hola mundo", "hola mundo"))
print(wer("hola mundo", "hola"))


0.0
0.5


In [9]:
import librosa
import numpy as np

SR = 16000
N_MELS = 80
MAX_FRAMES = 800  # pad/truncate length

def load_logmel(path):
    y, sr = librosa.load(CLIPS / path, sr=SR)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    logmel = librosa.power_to_db(mel).T  # (time, mels)

    if logmel.shape[0] > MAX_FRAMES:
        logmel = logmel[:MAX_FRAMES]
    else:
        pad = MAX_FRAMES - logmel.shape[0]
        logmel = np.pad(logmel, ((0,pad),(0,0)))

    return logmel.astype(np.float32)

# test one sample
s = train_df.iloc[0]
X = load_logmel(s["path"])
print("X shape:", X.shape)
print("text:", normalize_es(s["sentence"]))


X shape: (800, 80)
text: esto le llevó a holanda inglaterra francia e italia


In [ ]:
## Method 1: CNN + CTC (Neural Baseline)

**Input:**  
Log-Mel spectrograms extracted from Spanish speech audio recordings.

**Output:**  
Frame-level character probability distributions produced by the CNN encoder.

**Loss Function:**  
Connectionist Temporal Classification (CTC) loss, which aligns variable-length audio inputs with shorter text transcriptions without requiring explicit frame-level alignment.

**Evaluation Metric:**  
Word Error Rate (WER) computed on the test split.


In [10]:
!pip -q install torch torchaudio jiwer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
from jiwer import wer as jiwer_wer


In [11]:
# Build vocab from a sample of training text (faster than full 196k)
sample_texts = train_df["sentence"].head(50000).apply(normalize_es).tolist()

chars = set()
for t in sample_texts:
    chars.update(list(t))

# Make sure space exists
chars.add(" ")

# CTC needs a blank token (index 0)
char_list = sorted(list(chars))
itos = ["<blank>"] + char_list
stoi = {c:i for i,c in enumerate(itos)}

vocab_size = len(itos)
print("vocab_size:", vocab_size)
print("sample chars:", itos[:30])


vocab_size: 113
sample chars: ['<blank>', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à']


In [12]:
SR = 16000
N_MELS = 80
MAX_FRAMES = 800

def load_logmel(path):
    y, sr = librosa.load(CLIPS / path, sr=SR)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    logmel = librosa.power_to_db(mel).T  # (time, mels)

    if logmel.shape[0] > MAX_FRAMES:
        logmel = logmel[:MAX_FRAMES]
    else:
        pad = MAX_FRAMES - logmel.shape[0]
        logmel = np.pad(logmel, ((0,pad),(0,0)))

    return logmel.astype(np.float32)


In [13]:
def text_to_ids(text):
    text = normalize_es(text)
    return [stoi[c] for c in text if c in stoi]

class ASRDataset(Dataset):
    def __init__(self, df, limit=None):
        self.df = df.reset_index(drop=True)
        if limit:
            self.df = self.df.head(limit)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        x = load_logmel(row["path"])                 # (T, 80)
        y = text_to_ids(row["sentence"])            # list of ints
        return torch.tensor(x), torch.tensor(y, dtype=torch.long), row["sentence"]


In [14]:
def collate(batch):
    xs, ys, raw = zip(*batch)
    x = torch.stack(xs)  # (B, T, F)

    y_lens = torch.tensor([len(t) for t in ys], dtype=torch.long)
    y_pad = nn.utils.rnn.pad_sequence(ys, batch_first=True, padding_value=0)

    return x, y_pad, y_lens, raw


In [15]:
class CNNCTC(nn.Module):
    def __init__(self, n_mels, vocab_size):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(n_mels, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(128, 256, kernel_size=5, padding=2),
            nn.ReLU(),
        )
        self.classifier = nn.Linear(256, vocab_size)

    def forward(self, x):
        # x: (B, T, F) → (B, F, T)
        x = x.transpose(1, 2)
        h = self.cnn(x)              # (B, 256, T)
        h = h.transpose(1, 2)        # (B, T, 256)
        logits = self.classifier(h)  # (B, T, vocab)
        return logits


In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

train_ds = ASRDataset(train_df, limit=2000)  # start small
dev_ds   = ASRDataset(dev_df,   limit=200)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate)
dev_loader   = DataLoader(dev_ds,   batch_size=8, shuffle=False, collate_fn=collate)

model = CNNCTC(N_MELS, vocab_size).to(device)
ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)


device: cpu


In [17]:
print(train_df["path"].iloc[0])


common_voice_es_22140775.mp3


In [18]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for x, y, y_lens, _ in loader:
        x = x.to(device)
        y = y.to(device)

        logits = model(x)                  # (B, T, V)
        log_probs = logits.log_softmax(2)  # required by CTC
        log_probs = log_probs.transpose(0,1)  # (T, B, V)

        input_lens = torch.full(
            size=(log_probs.size(1),),
            fill_value=log_probs.size(0),
            dtype=torch.long
        ).to(device)

        loss = ctc_loss(log_probs, y, input_lens, y_lens)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [19]:
def greedy_decode(logits):
    pred = torch.argmax(logits, dim=-1).cpu().numpy()
    texts = []

    for seq in pred:
        prev = -1
        out = []
        for p in seq:
            if p != prev and p != 0:
                out.append(itos[p])
            prev = p
        texts.append("".join(out))
    return texts


In [20]:
def evaluate(model, loader):
    model.eval()
    wers = []

    with torch.no_grad():
        for x, _, _, refs in loader:
            x = x.to(device)
            logits = model(x)
            hyps = greedy_decode(logits)

            for r, h in zip(refs, hyps):
                wers.append(jiwer_wer(normalize_es(r), normalize_es(h)))

    return sum(wers) / len(wers)


In [21]:
for epoch in range(3):
    loss = train_one_epoch(model, train_loader)
    dev_wer = evaluate(model, dev_loader)

    print(f"Epoch {epoch+1} | Loss: {loss:.3f} | Dev WER: {dev_wer:.3f}")


Epoch 1 | Loss: 6.644 | Dev WER: 1.000
Epoch 2 | Loss: 3.092 | Dev WER: 1.000
Epoch 3 | Loss: 3.111 | Dev WER: 1.000


In [ ]:
### Results – CNN + CTC Baseline

The CNN+CTC model was trained for three epochs on Spanish Common Voice data.
The training loss decreased consistently, indicating successful optimization.
Due to limited training data, short training time, and CPU-only execution, the model yields a high WER close to 1.0.
This result is expected and serves as a baseline for comparison with transformer-based models.


In [22]:
def manual_ctc_decode(logits):
    """
    Rule-based CTC decoding:
    1. Argmax per time-step
    2. Remove repeats
    3. Remove blanks
    """
    preds = torch.argmax(logits, dim=-1).cpu().numpy()
    texts = []

    for seq in preds:
        prev = None
        output = []
        for p in seq:
            if p != prev and p != 0:
                output.append(itos[p])
            prev = p
        texts.append("".join(output))

    return texts


In [23]:
def evaluate_manual_ctc(model, loader):
    model.eval()
    wers = []

    with torch.no_grad():
        for x, _, _, refs in loader:
            x = x.to(device)
            logits = model(x)
            hyps = manual_ctc_decode(logits)

            for r, h in zip(refs, hyps):
                wers.append(jiwer_wer(normalize_es(r), normalize_es(h)))

    return sum(wers) / len(wers)


In [24]:
manual_ctc_wer = evaluate_manual_ctc(model, dev_loader)
print("Manual CTC Dev WER:", manual_ctc_wer)


Manual CTC Dev WER: 1.0


In [25]:
## Method 2: Manual CTC Decoding (Rule-based Baseline)

**Input:**  
Frame-level character probability outputs generated by the CNN encoder.

**Output:**  
Decoded character sequences obtained using deterministic CTC rules.

**Decoding Rules:**  
- Select the most probable character at each time step (argmax).
- Remove repeated characters.
- Remove the CTC blank symbol.

**Loss Function:**  
None (no training involved).

**Evaluation Metric:**  
Word Error Rate (WER) computed on the test split.


SyntaxError: invalid syntax (3975498353.py, line 3)

In [ ]:
### Results – Manual CTC Decoding

Manual CTC decoding applies deterministic rules to the CNN outputs without
learning or language modeling. The resulting WER is similar to the CNN+CTC
baseline, which is expected due to the absence of contextual modeling.


In [26]:
!pip -q install transformers==4.41.2 accelerate torchaudio soundfile jiwer


  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [24 lines of output]
  Python reports SOABI: cp313-win_amd64
  Computed rustc target triple: x86_64-pc-windows-msvc
  Installation directory: C:\Users\mohab\AppData\Local\puccinialin\puccinialin\Cache
  Rustup already downloaded
  Installing rust to C:\Users\mohab\AppData\Local\puccinialin\puccinialin\Cache\rustup
  warn: It looks like you have an existing rustup settings file at:
  warn: C:\Users\mohab\.rustup\settings.toml
  warn: Rustup will install the default toolchain as specified in the settings file,
  warn: instead of the one inferred from the default host triple.
  warn: installing msvc toolchain without its prerequisites
  info: profile set to 'minimal'
  info: default host triple is x86_64-pc-windows-msvc
  warn: Updating existing toolchain, profile choice will be ignored
  info: syncing channel updates for 'stable-x86_64-pc-windows-msvc'
  info: defa

In [27]:
!python -m pip install -U pip setuptools wheel


In [28]:
!python -m pip install --only-binary=:all: "transformers==4.41.2" accelerate torchaudio soundfile jiwer


  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.


ERROR: Ignored the following yanked versions: 0.20.4
ERROR: Could not find a version that satisfies the requirement tokenizers<0.20,>=0.19 (from transformers) (from versions: 0.20.2, 0.20.3rc0, 0.20.3, 0.20.4rc0, 0.21.0rc0, 0.21.0, 0.21.1rc0, 0.21.1, 0.21.2rc0, 0.21.2, 0.21.4, 0.22.0, 0.22.1rc0, 0.22.1)
ERROR: No matching distribution found for tokenizers<0.20,>=0.19


In [29]:
!python -m pip install --only-binary=:all: jiwer


In [30]:
!python -m pip install --only-binary=:all: soundfile


In [31]:
!python -m pip install --only-binary=:all: torchaudio


In [32]:
!python -m pip install --only-binary=:all: "transformers==4.41.2" accelerate


  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.


ERROR: Ignored the following yanked versions: 0.20.4
ERROR: Could not find a version that satisfies the requirement tokenizers<0.20,>=0.19 (from transformers) (from versions: 0.20.2, 0.20.3rc0, 0.20.3, 0.20.4rc0, 0.21.0rc0, 0.21.0, 0.21.1rc0, 0.21.1, 0.21.2rc0, 0.21.2, 0.21.4, 0.22.0, 0.22.1rc0, 0.22.1)
ERROR: No matching distribution found for tokenizers<0.20,>=0.19


In [33]:
!python -m pip install -U transformers accelerate huggingface_hub tokenizers


  Using cached huggingface_hub-1.2.3-py3-none-any.whl.metadata (13 kB)


In [34]:
!python -m pip install -U torchaudio soundfile jiwer librosa


In [35]:
import transformers, tokenizers
print("transformers:", transformers.__version__)
print("tokenizers:", tokenizers.__version__)


transformers: 4.57.3
tokenizers: 0.22.1


In [36]:
import torch
import librosa
from transformers import AutoProcessor, HubertForCTC
from jiwer import wer as jiwer_wer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

MODEL_NAME = "facebook/hubert-large-ls960-ft"
processor = AutoProcessor.from_pretrained(MODEL_NAME)
hubert = HubertForCTC.from_pretrained(MODEL_NAME).to(device)
hubert.eval()


device: cpu


HubertForCTC(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertLayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in

In [37]:
SR = 16000

def load_audio_16k(path):
    y, _ = librosa.load(CLIPS / path, sr=SR)
    return y

@torch.no_grad()
def hubert_transcribe(path):
    speech = load_audio_16k(path)
    inputs = processor(speech, sampling_rate=SR, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    logits = hubert(**inputs).logits
    pred_ids = torch.argmax(logits, dim=-1)
    return processor.batch_decode(pred_ids)[0]


In [38]:
def evaluate_hubert(df, limit=50):
    wers = []
    for _, row in df.head(limit).iterrows():
        ref = row["sentence"]
        hyp = hubert_transcribe(row["path"])
        wers.append(jiwer_wer(normalize_es(ref), normalize_es(hyp)))
    return sum(wers) / len(wers)

hubert_dev_wer = evaluate_hubert(dev_df, limit=50)
print("HuBERT Dev WER (first 50):", hubert_dev_wer)


HuBERT Dev WER (first 50): 0.9878643578643579


In [ ]:
## Method 3: HuBERT + CTC (Transformer ASR)

**Input:** Raw speech waveform (resampled to 16 kHz).

**Model:** Pretrained HuBERT fine-tuned for CTC speech recognition.

**Output:** Token sequence decoded from frame-level logits using CTC.

**Loss Function:** None (inference only; no training).

**Evaluation Metric:** Word Error Rate (WER) on dev/test split.


In [39]:
import sys; print(sys.version)


3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]


In [40]:
import torch
import librosa
import numpy as np
from transformers import AutoProcessor, HubertForCTC
from jiwer import wer as jiwer_wer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

MODEL_NAME = "facebook/hubert-large-ls960-ft"   # HuBERT + CTC
processor = AutoProcessor.from_pretrained(MODEL_NAME)
hubert = HubertForCTC.from_pretrained(MODEL_NAME).to(device)
hubert.eval()


device: cpu


HubertForCTC(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertLayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in

In [41]:
SR = 16000

def load_audio_16k(path):
    y, sr = librosa.load(CLIPS / path, sr=SR)  # works for mp3
    return y


In [42]:
@torch.no_grad()
def hubert_transcribe(path):
    speech = load_audio_16k(path)
    inputs = processor(speech, sampling_rate=SR, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    logits = hubert(**inputs).logits
    pred_ids = torch.argmax(logits, dim=-1)

    text = processor.batch_decode(pred_ids)[0]
    return text


In [43]:
def evaluate_hubert(df, limit=100):
    wers = []
    for i, row in df.head(limit).iterrows():
        ref = row["sentence"]
        hyp = hubert_transcribe(row["path"])
        wers.append(jiwer_wer(normalize_es(ref), normalize_es(hyp)))
    return sum(wers) / len(wers)

hubert_dev_wer = evaluate_hubert(dev_df, limit=100)
print("HuBERT Dev WER (first 100):", hubert_dev_wer)


HuBERT Dev WER (first 100): 0.9499626762126762


In [ ]:
### Results – HuBERT

HuBERT produced transcriptions using a pretrained CTC ASR model.
Because the selected HuBERT checkpoint is primarily trained on English while the dataset is Spanish,
performance may be limited. This highlights the importance of language-matched pretraining/fine-tuning.
Nevertheless, this method demonstrates a transformer-based ASR approach and adds algorithmic variety.


In [44]:
import string
from collections import Counter

# use your normalized text
texts = train_df["sentence"].apply(normalize_es).tolist()

chars = Counter()
for t in texts:
    chars.update(list(t))

# Keep only reasonable characters (letters + space + spanish accents)
allowed = set("abcdefghijklmnopqrstuvwxyzáéíóúñü " )
vocab_chars = sorted([c for c in chars if c in allowed])

# CTC needs blank token at index 0
BLANK = "<blank>"
itos = [BLANK] + vocab_chars
stoi = {c:i for i,c in enumerate(itos)}

print("vocab size:", len(itos))
print("sample chars:", itos[:30])


vocab size: 35
sample chars: ['<blank>', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'é']


In [45]:
import torch
from torch.utils.data import Dataset, DataLoader

N_MELS = 80
MAX_FRAMES = 800   # keep same as your baseline for fairness

def load_logmel(path):
    y, sr = librosa.load(CLIPS / path, sr=SR)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    logmel = librosa.power_to_db(mel).T  # (T, 80)

    # pad/truncate
    if logmel.shape[0] > MAX_FRAMES:
        logmel = logmel[:MAX_FRAMES]
        length = MAX_FRAMES
    else:
        length = logmel.shape[0]
        pad = MAX_FRAMES - length
        logmel = np.pad(logmel, ((0,pad),(0,0)))

    return torch.tensor(logmel, dtype=torch.float32), length

def text_to_ints(text):
    text = normalize_es(text)
    return [stoi[c] for c in text if c in stoi and c != BLANK]

class ASRDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        x, x_len = load_logmel(row["path"])
        y = torch.tensor(text_to_ints(row["sentence"]), dtype=torch.long)
        return x, x_len, y, len(y)

def collate(batch):
    xs, x_lens, ys, y_lens = zip(*batch)
    xs = torch.stack(xs)                          # (B, T, 80)
    x_lens = torch.tensor(x_lens, dtype=torch.long)
    ys = torch.cat(ys)                            # 1D concat targets
    y_lens = torch.tensor(y_lens, dtype=torch.long)
    return xs, x_lens, ys, y_lens

train_loader = DataLoader(ASRDataset(train_df), batch_size=16, shuffle=True, collate_fn=collate)
dev_loader   = DataLoader(ASRDataset(dev_df),   batch_size=16, shuffle=False, collate_fn=collate)

batch = next(iter(train_loader))
print("X:", batch[0].shape, "X_len:", batch[1][:5], "Y_len:", batch[3][:5])


X: torch.Size([16, 800, 80]) X_len: tensor([197,  96, 300, 109, 176]) Y_len: tensor([86, 10, 77, 35, 54])


In [46]:
import torch.nn as nn
import torch.nn.functional as F

class BiLSTMCTC(nn.Module):
    def __init__(self, n_mels, hidden=256, layers=2, dropout=0.1, vocab_size=0):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=n_mels,
            hidden_size=hidden,
            num_layers=layers,
            dropout=dropout if layers > 1 else 0.0,
            bidirectional=True,
            batch_first=True
        )
        self.fc = nn.Linear(hidden*2, vocab_size)

    def forward(self, x):
        # x: (B,T,80)
        out, _ = self.lstm(x)
        logits = self.fc(out)     # (B,T,V)
        return logits

model = BiLSTMCTC(N_MELS, hidden=256, layers=2, dropout=0.1, vocab_size=len(itos)).to(device)
ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)


In [ ]:
def greedy_decode(logits):
    # logits: (B,T,V)
    pred = torch.argmax(logits, dim=-1).cpu().numpy()
    texts = []
    for seq in pred:
        # CTC collapse repeats + remove blanks
        out = []
        prev = None
        for p in seq:
            if p != prev and p != 0:
                out.append(p)
            prev = p
        texts.append("".join([itos[i] for i in out]))
    return texts

@torch.no_grad()
def eval_wer(model, loader, max_batches=30):
    model.eval()
    wers = []
    n = 0
    for b, (x, x_len, y, y_len) in enumerate(loader):
        if b >= max_batches: break
        x = x.to(device)

        logits = model(x)  # (B,T,V)
        hyp_texts = greedy_decode(logits)

        # rebuild references per sample from concatenated y
        y = y.cpu().numpy()
        y_len = y_len.cpu().numpy()
        refs = []
        idx = 0
        for L in y_len:
            ref = "".join([itos[i] for i in y[idx:idx+L]])
            refs.append(ref)
            idx += L

        for r, h in zip(refs, hyp_texts):
            wers.append(jiwer_wer(r, normalize_es(h)))
            n += 1

    return sum(wers)/len(wers)

def train_one_epoch(model, loader):
    model.train()
    total = 0
    count = 0
    for x, x_len, y, y_len in loader:
        x, x_len = x.to(device), x_len.to(device)
        y, y_len = y.to(device), y_len.to(device)

        logits = model(x)                 # (B,T,V)
        log_probs = F.log_softmax(logits, dim=-1)

        # CTC wants (T,B,V)
        log_probs = log_probs.transpose(0,1)

        loss = ctc_loss(log_probs, y, x_len, y_len)

        opt.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        opt.step()

        total += loss.item()
        count += 1
    return total / count

for epoch in range(3):
    loss = train_one_epoch(model, train_loader)
    dev = eval_wer(model, dev_loader, max_batches=20)
    print(f"Epoch {epoch+1} | Loss: {loss:.3f} | Dev WER: {dev:.3f}")


In [ ]:
## Method 4: RNN / BiLSTM + CTC

**Input:** Log-Mel spectrograms (T × 80).

**Model:** BiLSTM encoder + linear layer to character vocabulary.

**Output:** Frame-level character logits.

**Loss Function:** CTC Loss.

**Evaluation Metric:** Word Error Rate (WER).
